# 变分自编码器

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.utils import save_image

In [13]:
# 定义超参数
image_size = 28*28
h_dim = 400
z_dim = 20
num_epoches = 30
batch_size = 128
learning_rate = 1e-3

torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [14]:
data_train = datasets.MNIST("./data", train=True, download=False, 
                            transform=transforms.ToTensor())
data_test = datasets.MNIST("./data", train=False, download=False, 
                            transform=transforms.ToTensor())

In [15]:
data_train_loader = DataLoader(dataset=data_train, batch_size=batch_size, 
                               shuffle=True)
data_test_loader = DataLoader(dataset=data_test, batch_size=batch_size, 
                               shuffle=True)

In [17]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc21 = nn.Linear(h_dim, z_dim)
        self.fc22 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        self.fc4 = nn.Linear(h_dim, image_size)
    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, image_size))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [21]:
model =  VAE().to(device)

In [22]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
def loss_func(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [29]:
log_interval = 10 # 每10个batch输出一次损失
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(data_train_loader):
        data = data.to(device)
        recon_x, mu, logvar = model(data)
        loss = loss_func(recon_x, data, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print("Trian Epoch:{} [{}/{} ({:.0f}%)] \tLoss:{:.6f}".format(
            epoch, batch_idx*len(data), len(data_train_loader.dataset), 
                100. * batch_idx / len(data_train_loader), loss.item() / len(data)))
        
        print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(data_train_loader.dataset)))

In [39]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(data_test_loader):
            data = data.to(device)
            recon_x, mu, logvar = model(data)
            test_loss += loss_func(recon_x, data, mu, logvar).item()
            # 比较第一个batch中真实图像与生成图像
            if i == 0: 
                n = min(len(data), 8) # 比较的图片数量
                comparison = torch.cat([data[:n], recon_x.view(-1, 1, 28, 28)[:n]])
                save_image(comparison.cpu(), "./results/reconstruction_" + str(epoch) + ".png", nrow=n)
            test_loss /= len(data_test_loader.dataset)
            print('====> Test set loss: {:.4f}'.format(test_loss))

In [40]:
def main():
    for epoch in range(1, num_epoches + 1):
        train(epoch)
        test(epoch)

In [41]:
main()

Trian Epoch:1 [0/60000 (0%)] 	Loss:111.212509
====> Epoch: 1 Average loss: 0.2373
====> Epoch: 1 Average loss: 0.4830
====> Epoch: 1 Average loss: 0.7240
====> Epoch: 1 Average loss: 0.9551
====> Epoch: 1 Average loss: 1.2013
====> Epoch: 1 Average loss: 1.4454
====> Epoch: 1 Average loss: 1.6871
====> Epoch: 1 Average loss: 1.9315
====> Epoch: 1 Average loss: 2.1688
====> Epoch: 1 Average loss: 2.4041
Trian Epoch:1 [1280/60000 (2%)] 	Loss:116.542191
====> Epoch: 1 Average loss: 2.6528
====> Epoch: 1 Average loss: 2.8977
====> Epoch: 1 Average loss: 3.1344
====> Epoch: 1 Average loss: 3.3730
====> Epoch: 1 Average loss: 3.6105
====> Epoch: 1 Average loss: 3.8532
====> Epoch: 1 Average loss: 4.0888
====> Epoch: 1 Average loss: 4.3362
====> Epoch: 1 Average loss: 4.5698
====> Epoch: 1 Average loss: 4.8118
Trian Epoch:1 [2560/60000 (4%)] 	Loss:109.059883
====> Epoch: 1 Average loss: 5.0445
====> Epoch: 1 Average loss: 5.2811
====> Epoch: 1 Average loss: 5.5299
====> Epoch: 1 Average loss:

====> Epoch: 1 Average loss: 47.0400
====> Epoch: 1 Average loss: 47.2712
====> Epoch: 1 Average loss: 47.5048
====> Epoch: 1 Average loss: 47.7520
====> Epoch: 1 Average loss: 47.9941
Trian Epoch:1 [25600/60000 (43%)] 	Loss:115.704575
====> Epoch: 1 Average loss: 48.2410
====> Epoch: 1 Average loss: 48.4751
====> Epoch: 1 Average loss: 48.7178
====> Epoch: 1 Average loss: 48.9517
====> Epoch: 1 Average loss: 49.1874
====> Epoch: 1 Average loss: 49.4393
====> Epoch: 1 Average loss: 49.6899
====> Epoch: 1 Average loss: 49.9290
====> Epoch: 1 Average loss: 50.1600
====> Epoch: 1 Average loss: 50.3960
Trian Epoch:1 [26880/60000 (45%)] 	Loss:113.240646
====> Epoch: 1 Average loss: 50.6376
====> Epoch: 1 Average loss: 50.8733
====> Epoch: 1 Average loss: 51.1132
====> Epoch: 1 Average loss: 51.3645
====> Epoch: 1 Average loss: 51.6014
====> Epoch: 1 Average loss: 51.8354
====> Epoch: 1 Average loss: 52.0748
====> Epoch: 1 Average loss: 52.3094
====> Epoch: 1 Average loss: 52.5473
====> Epoc

Trian Epoch:1 [49920/60000 (83%)] 	Loss:110.281158
====> Epoch: 1 Average loss: 93.5348
====> Epoch: 1 Average loss: 93.7698
====> Epoch: 1 Average loss: 94.0079
====> Epoch: 1 Average loss: 94.2517
====> Epoch: 1 Average loss: 94.4848
====> Epoch: 1 Average loss: 94.7148
====> Epoch: 1 Average loss: 94.9530
====> Epoch: 1 Average loss: 95.1862
====> Epoch: 1 Average loss: 95.4190
====> Epoch: 1 Average loss: 95.6544
Trian Epoch:1 [51200/60000 (85%)] 	Loss:117.428215
====> Epoch: 1 Average loss: 95.9049
====> Epoch: 1 Average loss: 96.1419
====> Epoch: 1 Average loss: 96.3820
====> Epoch: 1 Average loss: 96.6278
====> Epoch: 1 Average loss: 96.8679
====> Epoch: 1 Average loss: 97.1108
====> Epoch: 1 Average loss: 97.3503
====> Epoch: 1 Average loss: 97.5845
====> Epoch: 1 Average loss: 97.8198
====> Epoch: 1 Average loss: 98.0560
Trian Epoch:1 [52480/60000 (87%)] 	Loss:104.812012
====> Epoch: 1 Average loss: 98.2796
====> Epoch: 1 Average loss: 98.5137
====> Epoch: 1 Average loss: 98.7

====> Epoch: 2 Average loss: 14.8874
====> Epoch: 2 Average loss: 15.1264
====> Epoch: 2 Average loss: 15.3563
====> Epoch: 2 Average loss: 15.5923
====> Epoch: 2 Average loss: 15.8338
====> Epoch: 2 Average loss: 16.0630
====> Epoch: 2 Average loss: 16.2935
====> Epoch: 2 Average loss: 16.5366
Trian Epoch:2 [8960/60000 (15%)] 	Loss:108.262886
====> Epoch: 2 Average loss: 16.7676
====> Epoch: 2 Average loss: 17.0104
====> Epoch: 2 Average loss: 17.2489
====> Epoch: 2 Average loss: 17.4822
====> Epoch: 2 Average loss: 17.7199
====> Epoch: 2 Average loss: 17.9552
====> Epoch: 2 Average loss: 18.1955
====> Epoch: 2 Average loss: 18.4340
====> Epoch: 2 Average loss: 18.6721
====> Epoch: 2 Average loss: 18.9058
Trian Epoch:2 [10240/60000 (17%)] 	Loss:109.004890
====> Epoch: 2 Average loss: 19.1384
====> Epoch: 2 Average loss: 19.3684
====> Epoch: 2 Average loss: 19.5989
====> Epoch: 2 Average loss: 19.8368
====> Epoch: 2 Average loss: 20.0749
====> Epoch: 2 Average loss: 20.3165
====> Epoch

====> Epoch: 2 Average loss: 60.7477
====> Epoch: 2 Average loss: 60.9877
====> Epoch: 2 Average loss: 61.2228
Trian Epoch:2 [33280/60000 (55%)] 	Loss:109.761337
====> Epoch: 2 Average loss: 61.4570
====> Epoch: 2 Average loss: 61.6904
====> Epoch: 2 Average loss: 61.9274
====> Epoch: 2 Average loss: 62.1515
====> Epoch: 2 Average loss: 62.3883
====> Epoch: 2 Average loss: 62.6215
====> Epoch: 2 Average loss: 62.8540
====> Epoch: 2 Average loss: 63.0895
====> Epoch: 2 Average loss: 63.3233
====> Epoch: 2 Average loss: 63.5531
Trian Epoch:2 [34560/60000 (58%)] 	Loss:108.565857
====> Epoch: 2 Average loss: 63.7847
====> Epoch: 2 Average loss: 64.0103
====> Epoch: 2 Average loss: 64.2348
====> Epoch: 2 Average loss: 64.4669
====> Epoch: 2 Average loss: 64.7133
====> Epoch: 2 Average loss: 64.9417
====> Epoch: 2 Average loss: 65.1729
====> Epoch: 2 Average loss: 65.4032
====> Epoch: 2 Average loss: 65.6332
====> Epoch: 2 Average loss: 65.8704
Trian Epoch:2 [35840/60000 (60%)] 	Loss:110.497

...

====> Epoch: 18 Average loss: 79.9187
Trian Epoch:18 [46080/60000 (77%)] 	Loss:101.778732
====> Epoch: 18 Average loss: 80.1358
====> Epoch: 18 Average loss: 80.3664
====> Epoch: 18 Average loss: 80.5931
====> Epoch: 18 Average loss: 80.8138
====> Epoch: 18 Average loss: 81.0425
====> Epoch: 18 Average loss: 81.2606
====> Epoch: 18 Average loss: 81.4895
====> Epoch: 18 Average loss: 81.7087
====> Epoch: 18 Average loss: 81.9237
====> Epoch: 18 Average loss: 82.1408
Trian Epoch:18 [47360/60000 (79%)] 	Loss:100.603256
====> Epoch: 18 Average loss: 82.3554
====> Epoch: 18 Average loss: 82.5818
====> Epoch: 18 Average loss: 82.7990
====> Epoch: 18 Average loss: 83.0219
====> Epoch: 18 Average loss: 83.2528
====> Epoch: 18 Average loss: 83.4713
====> Epoch: 18 Average loss: 83.6892
====> Epoch: 18 Average loss: 83.9092
====> Epoch: 18 Average loss: 84.1328
====> Epoch: 18 Average loss: 84.3557
Trian Epoch:18 [48640/60000 (81%)] 	Loss:101.424362
====> Epoch: 18 Average loss: 84.5721
====> Ep

====> Epoch: 19 Average loss: 6.4182
====> Epoch: 19 Average loss: 6.6323
Trian Epoch:19 [3840/60000 (6%)] 	Loss:105.362488
====> Epoch: 19 Average loss: 6.8570
====> Epoch: 19 Average loss: 7.0778
====> Epoch: 19 Average loss: 7.3042
====> Epoch: 19 Average loss: 7.5283
====> Epoch: 19 Average loss: 7.7456
====> Epoch: 19 Average loss: 7.9787
====> Epoch: 19 Average loss: 8.1996
====> Epoch: 19 Average loss: 8.4227
====> Epoch: 19 Average loss: 8.6476
====> Epoch: 19 Average loss: 8.8708
Trian Epoch:19 [5120/60000 (9%)] 	Loss:104.897781
====> Epoch: 19 Average loss: 9.0946
====> Epoch: 19 Average loss: 9.3131
====> Epoch: 19 Average loss: 9.5402
====> Epoch: 19 Average loss: 9.7637
====> Epoch: 19 Average loss: 9.9829
====> Epoch: 19 Average loss: 10.2041
====> Epoch: 19 Average loss: 10.4146
====> Epoch: 19 Average loss: 10.6439
====> Epoch: 19 Average loss: 10.8711
====> Epoch: 19 Average loss: 11.0893
Trian Epoch:19 [6400/60000 (11%)] 	Loss:102.536911
====> Epoch: 19 Average loss: 

====> Epoch: 19 Average loss: 48.4606
====> Epoch: 19 Average loss: 48.6821
Trian Epoch:19 [28160/60000 (47%)] 	Loss:105.718384
====> Epoch: 19 Average loss: 48.9076
====> Epoch: 19 Average loss: 49.1256
====> Epoch: 19 Average loss: 49.3471
====> Epoch: 19 Average loss: 49.5660
====> Epoch: 19 Average loss: 49.7888
====> Epoch: 19 Average loss: 50.0114
====> Epoch: 19 Average loss: 50.2348
====> Epoch: 19 Average loss: 50.4582
====> Epoch: 19 Average loss: 50.6869
====> Epoch: 19 Average loss: 50.9143
Trian Epoch:19 [29440/60000 (49%)] 	Loss:102.431137
====> Epoch: 19 Average loss: 51.1328
====> Epoch: 19 Average loss: 51.3567
====> Epoch: 19 Average loss: 51.5881
====> Epoch: 19 Average loss: 51.8108
====> Epoch: 19 Average loss: 52.0281
====> Epoch: 19 Average loss: 52.2547
====> Epoch: 19 Average loss: 52.4725
====> Epoch: 19 Average loss: 52.6926
====> Epoch: 19 Average loss: 52.9108
====> Epoch: 19 Average loss: 53.1257
Trian Epoch:19 [30720/60000 (51%)] 	Loss:104.661690
====> Ep

====> Epoch: 19 Average loss: 90.4630
====> Epoch: 19 Average loss: 90.6876
====> Epoch: 19 Average loss: 90.9124
Trian Epoch:19 [52480/60000 (87%)] 	Loss:103.576080
====> Epoch: 19 Average loss: 91.1334
====> Epoch: 19 Average loss: 91.3583
====> Epoch: 19 Average loss: 91.5886
====> Epoch: 19 Average loss: 91.8066
====> Epoch: 19 Average loss: 92.0229
====> Epoch: 19 Average loss: 92.2426
====> Epoch: 19 Average loss: 92.4633
====> Epoch: 19 Average loss: 92.6817
====> Epoch: 19 Average loss: 92.9152
====> Epoch: 19 Average loss: 93.1373
Trian Epoch:19 [53760/60000 (90%)] 	Loss:102.590919
====> Epoch: 19 Average loss: 93.3562
====> Epoch: 19 Average loss: 93.5807
====> Epoch: 19 Average loss: 93.7937
====> Epoch: 19 Average loss: 94.0139
====> Epoch: 19 Average loss: 94.2393
====> Epoch: 19 Average loss: 94.4547
====> Epoch: 19 Average loss: 94.6753
====> Epoch: 19 Average loss: 94.8957
====> Epoch: 19 Average loss: 95.1150
====> Epoch: 19 Average loss: 95.3384
Trian Epoch:19 [55040/

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    sample = torch.randn(64, 20).to(device)
    sample = model.decode(sample).cpu()
    save_image(sample.view(64, 1, 28, 28),
               './results/sample_' + str(epoch) + '.png')